In [ ]:
# Authenticate github

from github import Github
import os
from dotenv import load_dotenv

# set directory of file to root directory
os.chdir("../")
load_dotenv()


# Replace with your GitHub personal access token
ACCESS_TOKEN = os.getenv("GITHUB_TOKEN")

# Authenticate
g = Github(ACCESS_TOKEN)

# Get user info
user = g.get_user()
print("User Login:", user.login)

# List repositories
for repo in user.get_repos():
    print(f"Repo: {repo.name}")


In [ ]:
import requests

# https://docs.github.com/en/graphql/reference/queries

# GitHub API endpoint
BASE_URL = "https://api.github.com/search/repositories"

# Search query
query = "python"
params = {
    "q": query,  # The search query
    "sort": "stars",  # Sort by stars
    "order": "desc",  # Descending order
    "per_page": 10,   # Number of results per page
}

# Optionally use your personal access token for authentication
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# Make the API request
response = requests.get(BASE_URL, params=params, headers=headers)

# Check the response
if response.status_code == 200:
    results = response.json()
    for repo in results["items"]:
        print(f"Name: {repo['name']}")
        print(f"URL: {repo['html_url']}")
        print(f"Stars: {repo['stargazers_count']}")
        print("-" * 40)
else:
    print(f"Failed to fetch data: {response.status_code}")
    print(response.json())


In [ ]:
import requests

# get repo based on query - modified chatgpt code
 
# uses query string

# GitHub API endpoint
BASE_URL = "https://api.github.com/search/code"


# Search parameters
query = "touchdesigner in:readme filename:README.md"  # Your query string
params = {
    "q": query,  # The search query
    "per_page": 10,  # Number of results per page
}

# Headers with authorization
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Accept": "application/vnd.github+json",
}

# Make the request
response = requests.get(BASE_URL, params=params, headers=headers)

# Check the response
if response.status_code == 200:
    results = response.json()
    for item in results.get("items", []):
        print(f"File: {item['name']}")
        print(f"Path: {item['path']}")
        print(f"Repository: {item['repository']['full_name']}")
        print(f"URL: {item['html_url']}")
        print("-" * 40)
else:
    print(f"Error: {response.status_code}")
    print(response.json())


In [ ]:
import requests
import base64

# get readme file content from query

# GitHub API endpoint template
BASE_URL = "https://api.github.com/repos/{owner}/{repo}/contents/{path}"

# Replace these with your repository details
owner = "microsoft"
repo = "Reporting-Services-LoadTest"
path = "Readme.md"

# Optionally use your personal access token for private repos or higher rate limits

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Accept": "application/vnd.github+json",
}

# Construct the URL
url = BASE_URL.format(owner=owner, repo=repo, path=path)

# Make the request
response = requests.get(url, headers=headers)

# Process the response
if response.status_code == 200:
    file_info = response.json()
    content_encoded = file_info.get("content")
    if content_encoded:
        # Decode the Base64 content
        content = base64.b64decode(content_encoded).decode("utf-8")
        print("README Content:\n")
        print(content)
    else:
        print("README file is empty or content not found.")
else:
    print(f"Failed to fetch README: {response.status_code}")
    print(response.json())


In [ ]:
import requests

# more complicated graphQL endpoint

# GitHub GraphQL API endpoint
GRAPHQL_URL = "https://api.github.com/graphql"

# Replace with your GitHub personal access token


# GraphQL query string
query_string = """
{
  search(query: "touchdesigner glsl in:readme", type: REPOSITORY, first: 10) {
    edges {
      node {
        ... on Repository {
          name
          owner {
            login
          }
          url
          description
        }
      }
    }
  }
}
"""

# Headers for authorization
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json",
}

# Make the request
response = requests.post(
    GRAPHQL_URL,
    json={"query": query_string},
    headers=headers
)

# Process the response
if response.status_code == 200:
    data = response.json()
    for edge in data["data"]["search"]["edges"]:
        node = edge["node"]
        print(f"Name: {node['name']}")
        print(f"Owner: {node['owner']['login']}")
        print(f"URL: {node['url']}")
        print(f"Description: {node['description']}")
        print("-" * 40)
else:
    print(f"Error: {response.status_code}")
    print(response.json())
